In [22]:
import scipy.io
from pyedflib import highlevel
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import groupby
import csv
import pickle
from scipy.signal import butter, sosfilt, sosfiltfilt, sosfreqz
from scipy.signal import freqz, iirnotch, filtfilt
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin, BaseEstimator

# Variables and loading data

In [23]:
sampling_rate = 256

In [24]:
# Load annotation file
annt = scipy.io.loadmat('../raw_data/annotations_2017.mat')

In [25]:
# Load the data 
signals, signal_headers, header = highlevel.read_edf('../raw_data/eeg2.edf')

# Functions to preprocess

In [26]:
# Highpass filter
def highpass_filter(signals, sampling_rate, hp_frequency = 0.1):
    sos = butter(N = 3, Wn = hp_frequency, btype="highpass",fs=sampling_rate, output="sos")
    filter_hp = sosfiltfilt(sos, signals)
    return filter_hp

In [27]:
# Powerline filter
def notch_filter(signals, sampling_rate, notch_frequency = 50, quality_factor = 30):
    w0 = notch_frequency/(sampling_rate/2)
    b_notch, a_notch = iirnotch(w0, quality_factor)
    filter_notch = filtfilt(b_notch, a_notch, signals, axis = -1)
    return filter_notch

In [58]:
# Create our own scaler
class CustomTranformer(TransformerMixin, BaseEstimator): 
    # BaseEstimator generates the get_params() and set_params() methods that all Pipelines require
    # TransformerMixin creates the fit_transform() method from fit() and transform()
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.means = X.mean()
        return self
    
    def transform(self, X, y=None):
        norm_features = X - self.means
        return norm_features

In [29]:
# Combination of all filters and Scaler
def filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30):
    filter_hp = highpass_filter(signals, sampling_rate)
    filter_notch = notch_filter(signals, sampling_rate, notch_frequency, quality_factor)
    final_signal = scaler.fit_transform(filter_notch)
    return final_signal

# Functions to label

In [30]:
# Format the EEG 
def eeg_formated(signals, names_ele):
    data_signals = signals.T # transpose the signals from datapoints
    data_signals = pd.DataFrame(data_signals) # create a pandas dataframe
    
    data_signals.columns = names_ele # rename columns
    
    return data_signals

In [31]:
# Format the annotations
def diagnosis(n):
    patient_A=annt["annotat_new"][0][n-1][0]
    patient_B=annt["annotat_new"][0][n-1][1]
    patient_C=annt["annotat_new"][0][n-1][2]
    
    #converting seconds to datapoints

    patient_A=patient_A.tolist()
    patient_B=patient_B.tolist()
    patient_C=patient_C.tolist()
    
    patient_A_dtp=[]
    patient_B_dtp=[]
    patient_C_dtp=[]  
    for elem in patient_A:
        for i in range(sampling_rate):
            patient_A_dtp.append(elem) 
    for elem in patient_B:
        for i in range(sampling_rate):
            patient_B_dtp.append(elem)
        
    for elem in patient_C:
        for i in range(sampling_rate):
            patient_C_dtp.append(elem)
            
    target_=pd.DataFrame({"Diagnosis A":patient_A_dtp,"Diagnosis B":patient_B_dtp,"Diagnosis C":patient_C_dtp})
    
    return target_  

In [32]:
# Add a time column with the seconds
def add_time(df):
    list_time=[]
    for i in range(len(df)):
        list_time.append(i//sampling_rate)
    df["time"]=list_time
    return df

In [33]:
# Create target variables when seizures lasts at least 10
def is_seizure(df):
    
    threshold = sampling_rate*10
    
    df['is_seizure_A'] = df["Diagnosis A"].groupby((df["Diagnosis A"] != df["Diagnosis A"].shift()).cumsum()).transform('size') * df["Diagnosis A"]
    df['is_seizure_A'] = (df['is_seizure_A'] > threshold).astype(int)
    
    df['is_seizure_B'] = df["Diagnosis B"].groupby((df["Diagnosis B"] != df["Diagnosis B"].shift()).cumsum()).transform('size') * df["Diagnosis B"]
    df['is_seizure_B'] = (df['is_seizure_B'] > threshold).astype(int)
    
    df['is_seizure_C'] = df["Diagnosis C"].groupby((df["Diagnosis C"] != df["Diagnosis C"].shift()).cumsum()).transform('size') * df["Diagnosis C"]
    df['is_seizure_C'] = (df['is_seizure_C'] > threshold).astype(int)
    
    return df 

In [34]:
# Create final target
def create_target(df):
    df['is_seizure_target'] = np.where(df['is_seizure_A'] + df['is_seizure_B'] + df['is_seizure_C'] >= 2, 1, 0)
    return df

In [35]:
# Remove useless
def remove_useless_columns(df):
    df.drop(columns=['Diagnosis A', 'Diagnosis B', 'Diagnosis C', 'is_seizure_A', 'is_seizure_B', 'is_seizure_C', 'ECG EKG', 'Resp Effort', 'time'], inplace=True)
    return df

In [52]:
# Final function to label
def label_data(path_raw_data, signals_preprocessed, n):
    
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    names_ele = [signal_headers[iele]['label'] for iele in range(signals.shape[0])] # extract electrode names
    
    eeg_patient = eeg_formated(signals_preprocessed, names_ele) # format the ECG
    eeg_patient.rename(columns={'ECG EKG-REF':'ECG EKG', 'Resp Effort-REF':'Resp Effort'}, inplace=True)
    diagnosis_patient = diagnosis(n) # format the diagnosis
    
    data_patient = pd.merge(left=eeg_patient, right=diagnosis_patient, how='left', left_index=True, right_index=True) # merge ecg and diagnosis
    
    add_time(data_patient)
    is_seizure(data_patient)
    create_target(data_patient)
    remove_useless_columns(data_patient)
    
    return data_patient

# Final preprocessing & label

In [37]:
# Preprocessing
signals_preprocessed = filter_signals(signals, sampling_rate, CustomTranformer(), hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30)

#  Label
d_2 = label_data('../raw_data/eeg2.edf', signals_preprocessed, 2)

d_2.head()

,EEG Fp1-Ref,EEG Fp2-Ref,EEG F3-Ref,EEG F4-Ref,EEG F7-Ref,EEG F8-Ref,EEG Fz-Ref,EEG C3-Ref,EEG C4-Ref,EEG Cz-Ref,EEG T3-Ref,EEG T5-Ref,EEG T4-Ref,EEG T6-Ref,EEG P3-Ref,EEG P4-Ref,EEG Pz-Ref,EEG O1-Ref,EEG O2-Ref,is_seizure_target
0,-98.645167,-101.167312,-150.805429,-99.376346,-83.182367,-74.788670,20.013543,-143.138792,-87.463916,-77.354371,-139.180264,-9.668076,-84.621197,-61.668935,-7.265924,-127.848488,-58.116788,-140.756929,-126.694843,0
1,-98.821001,-101.178549,-152.318462,-100.563842,-81.687831,-76.560804,20.228367,-146.428835,-89.349899,-78.247286,-139.842162,-11.709217,-63.372738,-63.000002,-9.963888,-129.148470,-61.498554,-143.333635,-129.172922,0
2,-97.067707,-99.467840,-151.524433,-99.432681,-81.461237,-77.183123,19.782924,-145.092980,-88.052963,-77.638940,-157.247211,-12.517739,-78.761883,-61.846963,-11.261941,-127.605515,-61.651990,-142.162175,-126.797940,0
3,-98.678719,-101.018678,-153.656575,-102.018683,-83.169448,-79.906383,19.882898,-147.323366,-89.776689,-78.882967,-138.599045,-10.345296,-68.253342,-63.357444,-11.514185,-130.261691,-62.997133,-144.038379,-128.631423,0
4,-97.898545,-100.469111,-151.807918,-101.011273,-80.232267,-79.792650,19.633745,-145.928515,-88.713656,-77.980124,-144.491543,-10.629773,-71.803903,-64.094909,-12.265190,-128.846900,-63.588286,-142.847917,-128.002320,0


In [55]:
# Exclude patients without seizures

patients_numbers = list(range(1, 80))

patient_with_issue = [4, 29, 50] # Can't import ECG4, ECG29 and ECG50
patient_without_seizure = [3, 6, 10, 12, 18, 24, 26, 27, 28, 30, 32, 35, 37, 42, 43, 45, 46, 48, 49, 53, 55, 56, 57, 58, 59, 60, 61, 64, 65, 70, 72, 74]

for i in patient_with_issue:
    patients_numbers.remove(i)

for i in patient_without_seizure:
    patients_numbers.remove(i)

In [56]:
# MinMaxScaler
    
for i in patients_numbers:
    signals, signal_headers, header = highlevel.read_edf(f'../raw_data/eeg{i}.edf')
    signals_preprocessed_i = filter_signals(signals, sampling_rate, MinMaxScaler(), hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30)
    d_i = label_data(f'../raw_data/eeg{i}.edf', signals_preprocessed_i, i)
    d_i.to_csv(f'../data_modeling/MinMaxScaler/data_modeling_patient_{i}.csv')
    

In [ ]:
# CustomTransformer
    
for i in patients_numbers:
    signals, signal_headers, header = highlevel.read_edf(f'../raw_data/eeg{i}.edf')
    signals_preprocessed_i = filter_signals(signals, sampling_rate, CustomTranformer(), hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30)
    d_i = label_data(f'../raw_data/eeg{i}.edf', signals_preprocessed_i, i)
    d_i.to_csv(f'../data_modeling/CustomTransformer/data_modeling_patient_{i}.csv')
    

# Label only patients with seizures

In [54]:
for i in patients_numbers:
    d_i = label_data(f'../raw_data/eeg{i}.edf', signals, i)
    d_i.to_csv(f'../data_label/data_patient_{i}.csv')